In [0]:
import logging
from src.data_transformers import DataTransformers
from src.save_data import SaveData


def job(logger):
    select_list = [ "vendor_id", "passenger_count", "total_amount", "tpep_pickup_datetime", "tpep_dropoff_datetime", "year_month"]
    df = spark.table('ifood_case.bronze.yellow_trip')
    
    transformacao_silver = DataTransformers(
        logger=logger,
        df_spark=df,
        column_select=select_list,
        partition_column='tpep_pickup_datetime',
        partition_name="year_month",
        filter_column='year_month',
        filter_list=['2023-01', '2023-02', '2023-03', '2023-04', '2023-05'],
    )

    df_silver = transformacao_silver.filter_dataframe_spark()
    df_silver = transformacao_silver.select_expr_dataframe_spark()

    save_silver = SaveData(
        logger=logger,
        spark=spark,
        df=df_silver,
        layer='silver',
        table='yellow_trip',
        mode_save='append',
        format_save='delta',
        partition='year_month'
    )

    save_silver.saveData()


if __name__ == "__main__":
    logger = logging.getLogger("silver")
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(name)s - %(message)s"
    )

    job(logger)